In [ ]:
import snowoptics

import numpy as np
import pandas as pd

%matplotlib widget
import matplotlib.pyplot as plt

In [ ]:
# generate albedo

slope = np.deg2rad(10)
aspect = np.deg2rad(130)

ssa = 20

lon, lat = 0, 45
dt = pd.date_range('2020-03-22 08:00:00', '2020-03-22 17:00:00', freq='1H').values
sza, saa = snowoptics.compute_sun_position(lon, lat, dt)
print(np.cos(sza))

wavelengths = np.arange(400, 1100, 5) * 1e-9
r_difftot = 0.7 * (400e-9 / wavelengths) ** 3.5

albedo = [snowoptics.albedo_P20_slope(wavelengths, sza_, saa_, ssa, r_difftot, slope, aspect, "small_slope") for sza_, saa_ in zip(sza, saa)]

In [ ]:
wls_nm = wavelengths * 1e9

plt.figure(figsize=(10, 8))
for alb in albedo:
    plt.plot(wls_nm, alb)
    
plt.plot(wls_nm, snowoptics.albedo_diffuse_KZ04(wavelengths, ssa), 'k-', lw=2)

In [ ]:
# apply the unconstrained correction

albedo_diff, slope, aspect = snowoptics.albedo_timeseries_correction(wavelengths, albedo, r_difftot, sza, saa, constrained=False)

plt.plot(wls_nm, albedo_diff, 'x--', alpha=0.4, label="unconstrained correction")
plt.legend()
np.rad2deg(slope), np.rad2deg(aspect)

In [ ]:
# apply the constrained correction

albedo_diff, slope, aspect = snowoptics.albedo_timeseries_correction(wavelengths, albedo, r_difftot, sza, saa, constrained=True)

plt.plot(wls_nm, albedo_diff, '^--', alpha=0.4, label="unconstrained correction")
plt.legend()
np.rad2deg(slope), np.rad2deg(aspect)